In [149]:
!pip install optuna

In [150]:
from google.colab import drive
drive.mount('/content/drive')

#import required packages
import os
import pandas as pd
from scipy import stats
from numpy import mean
import numpy as np
import glob
from sklearn import metrics
from sklearn.metrics import roc_curve
from lightgbm import LGBMClassifier
import optuna
from collections import defaultdict
RANDOM_SEED = 5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [169]:
def sort_tuple(tup,i,rev = False):
  '''given a tuple, sorts it based on index i'''
  for x in tup:
    assert isinstance(x[i], float)
  tup.sort(key = lambda x: x[i],reverse = rev)
  return tup

In [170]:
def get_dummies(df,cols):
  '''create one-hot encoding of given columns'''
  assert isinstance(cols, list)
  assert isinstance(df, pd.DataFrame)
  df = pd.get_dummies(df, columns=[col for col in ['genderCAP','racecodeCAP']])
  return df

In [153]:
def normalize_cols(df,cols_to_norm):
  '''normalise the given columns'''
  assert isinstance(df, pd.DataFrame)
  assert isinstance(cols_to_norm, list)
  df[cols_to_norm]= df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max()-x.min()))
  return df

In [154]:
def remove_consecutive_rnfl(model):
  '''removes consecutive rnfl features as they are highly correlated'''
  assert isinstance(model, list)
  for feat in model:
    if 'rnfl_sec' in feat:
      number = feat.split('_')[2]

      # define the neighboring rnfl sectors
      plus1 = 'rnfl_sec_'+str(int(number)+1)+'_slope'
      minus1 = 'rnfl_sec_'+str(int(number)-1)+'_slope'
      if plus1 in model:
        model.remove(plus1)
      if minus1 in model:
        model.remove(minus1)
  return model

In [155]:
def remove_consecutive_gcc(model):
  '''removes consecutive gcc features as they are highly correlated'''
  assert isinstance(model, list)
  for feat in model:
    if 'gcc' in feat:
      number = feat.split('_')[0][-2:]

      # define the neighboring gcc sectors
      plus1 = 'mean'+str(int(number)+1)+'_gcc_slope'
      minus1 = 'mean'+str(int(number)+10)+'_gcc_slope'
      plus2 = 'mean'+str(int(number)-1)+'_gcc_slope'
      minus2 = 'mean'+str(int(number)-10)+'_gcc_slope'
      if plus1 in model:
        model.remove(plus1)
      if minus1 in model:
        model.remove(minus1)
      if plus2 in model:
        model.remove(plus2)
      if minus2 in model:
        model.remove(minus2)
  return model

In [156]:
def get_sorted_features(feat_df,group,y_var):
  '''sorts the features by importance (descending order),
     basing on their ability to discriminate the target
     variable, uses Kolmogorov-Smirov test'''
  assert isinstance(feat_df, pd.DataFrame)
  assert isinstance(y_var, str)

  # split data into positive and negative examples
  eve0 = feat_df[feat_df[y_var]==0]
  eve1 = feat_df[feat_df[y_var]==1]
  ks_tup_list = []

  for col in feat_df.columns:
    x = eve0[col]
    y = eve1[col]
    #for each feature, save the KS test stat
    ks_tup_list.append((col,stats.ks_2samp(x, y)[1]))

  # sort the features using their KS test stat
  ks_sorted = sort_tuple(ks_tup_list,1)
  best_feat_list = list(list(zip(*ks_sorted))[0])
  best_feat_list.remove(y_var)
  return best_feat_list

In [157]:
def remove_correlated_vars(list_of_models):
  '''removes correlated variables in the candidate model,
     calls both helper functions remove_consecutive_rnfl
     and remove_consecutive_gcc'''
  assert isinstance(list_of_models, list)
  new = []
  for model in list_of_models:
    assert isinstance(model, list)
    # for each candidate model, remove the neighboring gcc and rnfl features
    model = remove_consecutive_rnfl(model)
    model = remove_consecutive_gcc(model)
    if model not in new:
      new.append(model)
  return new

In [158]:
def get_best_clin(feat_df,group,y_var):
  '''gets the best clinical features using KS test'''
  assert isinstance(feat_df, pd.DataFrame)
  assert isinstance(y_var, str)
  eve0 = feat_df[feat_df[y_var]==0]
  eve1 = feat_df[feat_df[y_var]==1]
  best_feat_list = []
  ks_tup_list = []
  for col in feat_df.columns:
    x = eve0[col]
    y = eve1[col]
    ks_tup_list.append((col,stats.ks_2samp(x, y)[1]))

  ks_sorted = sort_tuple(ks_tup_list,1)
  best_feat_list = list(list(zip(*ks_sorted))[0])
  best_feat_list.remove(y_var)
  return best_feat_list

In [159]:
def get_list_of_models(min_non_clinical_features, clinical_features):
  '''splits the features into clinical and non-clinical (based on the name).
  then generates candidate models, each candidate model has 5 clinical features
  and at least x non-clinical features'''
  assert isinstance(min_non_clinical_features, int)
  assert isinstance(clinical_features, int)
  global trains, vals, test
  train_df = trains[0]
  test_df = test
  val_df = vals[0]

  gcc_rnfl,clin = split_features(train_df,val_df,'event')
  list_of_models = []
  base = min_non_clinical_features
  for j in [1,2,3]:
    a,b,c,d,e = base, base+2,base+4,base+6, base+8
    if j==1:
      model = gcc_rnfl[:a]+clin[:clinical_features]
    if j==2:
      model = gcc_rnfl[:b]+clin[:clinical_features]
    if j==3:
      model = gcc_rnfl[:c]+clin[:clinical_features]
    if j==4:
      model = gcc_rnfl[:d]+clin[:clinical_features]
    if j==5:
      model = gcc_rnfl[:e]+clin[:clinical_features]

    list_of_models.append(model)
  return list_of_models

In [160]:
def split_features(train_df,val_df,y_var):
  '''takes the train and val dfs, merges them and returns clinical and
  non-clinical features as lists (in descending order of importance)'''
  assert isinstance(train_df, pd.DataFrame)
  assert isinstance(val_df, pd.DataFrame)
  assert isinstance(y_var, str)
  feat_df = pd.concat([train_df,val_df])
  non_features = [col for col in feat_df.columns if 'rnflmean' in col]
  if y_var=='event':
    non_features += ['eye_id','sphere_equiv.1','PatientID','prog_all','prog_trend','prog_plr']
  elif y_var == 'prog_trend':
    non_features += ['eye_id','sphere_equiv.1','PatientID','prog_all','event','prog_plr']
  features = [col for col in feat_df.columns if col not in non_features]
  gcc = [col for col in features if 'gcc' in col]
  rnfl = [col for col in features if 'rnfl' in col]
  clin = ['iopmeanfu','iopundil_meas','genderCAP_Male','racecodeCAP_Black or African American',\
          'iopmaxfu','iopminfu','sphere_equiv','diabetesCAP','systolic','diastolic','md_24','psd_24']
  group = gcc+rnfl
  group.append(y_var)
  df = feat_df[group]
  group_best = get_sorted_features(df,group,y_var)
  clin.append(y_var)
  df2 = feat_df[clin]
  clin = get_best_clin(df2,clin,y_var)
  return group_best,clin

In [161]:
def train_and_test_final_model(final_feat, final_params):
  '''takes the best features and best hyper-parameters (given by the val set),
  then retrains the model on the combined train and val set,
  then performs testing on the test set'''
  assert isinstance(final_feat, list)

  global trains, vals, test

  train_df = trains[0]
  test_df = test
  val_df = vals[0]

  train_df = normalize_cols(train_df,final_feat)
  test_df = normalize_cols(test_df,final_feat)
  val_df = normalize_cols(val_df,final_feat)

  x_train,y_train = train_df[final_feat],train_df['event']
  x_val,y_val = val_df[final_feat],val_df['event']
  x_test,y_test = test_df[final_feat],test_df['event']

  x_train_val,y_train_val = pd.concat([x_train,x_val],ignore_index=True),pd.concat([y_train,y_val])
  model = LGBMClassifier(**final_params)#,is_unbalance=True

  model.fit(x_train_val, y_train_val)
  pred_probs = model.predict_proba(x_test)
  auc = metrics.roc_auc_score(y_test, pred_probs[:,1])
  fpr,tpr, _ = roc_curve(y_test, pred_probs[:,1])
  roc_data = list(zip(fpr,tpr))

  test_df['y_hat'] = pred_probs[:,1]
  boots = test_df[['PatientID','event','y_hat','eye_id']]
  boots.rename(columns={'event': 'y_true', 'PatientID': 'patient_ID','eye_id':'eye_ID'}, inplace=True)

  return (auc,roc_data,boots)

In [162]:
def get_optuna_model(min_non_clinical_features, clinical_features):
  '''for each candidate model, get the best hyperparameters and
  performance on validation set'''
  assert isinstance(min_non_clinical_features,  int)
  assert isinstance(clinical_features, int)
  list_of_models = get_list_of_models(min_non_clinical_features, clinical_features)
  list_of_models = remove_correlated_vars(list_of_models)

  all_results = []
  for candidate_model in list_of_models:
    cw = {0:0.3,1:0.7}
    candi_res = get_res(candidate_model)
    all_results.append(candi_res)

  return all_results


In [163]:
def process_and_save_data(path):
  '''load all the required csvs and save them in global variables,
  this helps avoid repeated read calls'''
  assert os.path.exists(path)
  global trains, vals, test

  test_df = pd.read_csv(path + 'event_test'+'.csv')
  test_df = test_df.loc[:, ~test_df.columns.str.contains('^Unnamed')]
  test = test_df
  for i in range(4):
      train_df = pd.read_csv(path + 'event_train_'+str(i+1)+'.csv' )
      val_df = pd.read_csv(path + 'event_val_'+str(i+1)+'.csv' )
      train_df = train_df.loc[:, ~train_df.columns.str.contains('^Unnamed')]
      val_df = val_df.loc[:, ~val_df.columns.str.contains('^Unnamed')]
      trains.append(train_df)
      vals.append(val_df)

In [164]:
def get_res(all_feat):
  '''runs the experiments on hyperparameters using optuna, records and returns
  the best validation auc and the corresponding hyperparmeters for a
  candidate model. The validation auc is deteremined by 4-fold cross val'''
  assert isinstance(final_feat, list)
  global trains, vals, test

  all_feat = all_feat
  def objective(trial):
    n_estimators_hp = trial.suggest_int('n_estimators', 50,500, step = 50)
    max_depth_hp = trial.suggest_int('max_depth', 2,7)
    min_child_samples_hp = trial.suggest_int('min_child_samples', 10, 50, step = 10)
    num_leaves_hp = trial.suggest_int('num_leaves', 2, 8)
    learning_rate_hp = trial.suggest_float('learning_rate',1e-4,10)
    max_bin_hp = trial.suggest_int('max_bin',5,10)
    lambda_l2_hp = trial.suggest_float('reg_lambda',0,2)
    bagging_fraction_hp = trial.suggest_float('subsample',1e-4,1)

    roc_val_li = []

    for i in range(4):
          train_df = trains[i]
          test_df = test
          val_df = vals[i]

          train_df = normalize_cols(train_df,all_feat)
          test_df = normalize_cols(test_df,all_feat)
          val_df = normalize_cols(val_df,all_feat)

          x_train,y_train = train_df[all_feat],train_df['event']
          x_val,y_val = val_df[all_feat],val_df['event']
          x_test,y_test = test_df[all_feat],test_df['event']

          x_train_val,y_train_val = pd.concat([x_train,x_val],ignore_index=True),pd.concat([y_train,y_val])
          cw = {0:0.3,1:0.7}
          model = LGBMClassifier(n_estimators = n_estimators_hp,
                                max_depth = max_depth_hp,
                                min_child_samples = min_child_samples_hp,
                                num_leaves = num_leaves_hp,
                                learning_rate = learning_rate_hp,
                                max_bin = max_bin_hp,
                                reg_lambda = lambda_l2_hp,
                                subsample = bagging_fraction_hp,
                                random_state = 0,is_unbalance=True)#False, class_weight = cw

          model.fit(x_train, y_train)
          pred_probs = model.predict_proba(x_val)

          roc_val_li.append(metrics.roc_auc_score(y_val, pred_probs[:,1]))

    return sum(roc_val_li)/len(roc_val_li)

  model_res = defaultdict(dict)
  model_res['features'] = all_feat
  model_res['name'] = 'test set'

  optuna.logging.set_verbosity(optuna.logging.FATAL)

  study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=2022))
  study.optimize(objective, n_trials=50)

  model_res['best_para'] = study.best_trial.params
  model_res['avg_val_auc'] = study.best_trial.value

  return model_res

In [165]:
def write_model_to_file(best_model,outpath,outfile):
  '''writes the final model to a text file'''
  assert isinstance(best_model, list)
  assert isinstance(outfile, str)
  assert os.path.exists(outpath)
  file_path = '/content/drive/Shareddrives/Glaucoma_features/k198/train_val_test_splits/'+outfile
  if os.path.exists(file_path):
    with open(file_path, 'a') as file:
        file.write(str(best_model) + '\n')
  else:
    with open(file_path, 'w') as file:
        file.write(str(best_model) + '\n')

In [166]:
def get_final_model_for_test_set(list_of_all_results):
  '''determines the best features and the corresponding hyperparameters,
  basing on the average validation auc'''
  assert isinstance(list_of_all_results, list)
  key = "avg_val_auc"

  # Finding the dictionary with the highest value for the given key
  best_for_given_subset = max(list_of_all_results, key=lambda x: x.get(key, 0))
  print(best_for_given_subset['best_para'])
  return best_for_given_subset['features'], best_for_given_subset['best_para']


In [167]:
%%capture
trains = []
vals = []
test = None

# set the min. number of non-clinical features
min_non_clinical_features = 13
clinical_features = 5
aucs_m = []

# specify input and output paths
output_path = '/content/drive/Shareddrives/Glaucoma_features/k198/Raghu_Optuna/'
input_path = '/content/drive/Shareddrives/Glaucoma_features/k198/train_val_test_splits/'

# cache
process_and_save_data(input_path)
list_of_set_results = get_optuna_model(min_non_clinical_features, clinical_features)
final_feat, final_params = get_final_model_for_test_set(list_of_set_results)
auc_i, roc_i, boots = train_and_test_final_model(final_feat, final_params)

# save the results
write_model_to_file(final_feat,output_path,outfile='LGBM.txt')
boots.to_csv(output_path+'boots'+'.csv')
aucs_m.append(auc_i)
print(aucs_m)



In [168]:
aucs_m

[0.7875]